# Input several news articles into the Stardog DNA repositories

There are 8 articles from different sources on the topics of the UN Gaza Ceasefire vote, the lowering of the Trump bond payment, the start of the Trump NY trial in mid-April, and the FL Supreme Court upholding the 6-week abortion ban.

Make sure that: 
* The DNA flask is running (execute 'flask run' from the dna directory)

In [1]:
import json
import requests

In [2]:
# Delete repositories (if they exist)
response = requests.delete('http://127.0.0.1:5000/dna/v1/repositories?repository=ceasefire')
print(response.status_code, response) 
response = requests.delete('http://127.0.0.1:5000/dna/v1/repositories?repository=trump_bond')
print(response.status_code, response) 
response = requests.delete('http://127.0.0.1:5000/dna/v1/repositories?repository=fl_abortion')
print(response.status_code, response) 
response = requests.delete('http://127.0.0.1:5000/dna/v1/repositories?repository=trump_trial')
print(response.status_code, response) 

404 <Response [404]>
404 <Response [404]>
200 <Response [200]>
200 <Response [200]>


In [3]:
# Create the repositories for the news articles
# response = requests.post('http://127.0.0.1:5000/dna/v1/repositories?repository=ceasefire')
# print(response.status_code, response) 
# response = requests.post('http://127.0.0.1:5000/dna/v1/repositories?repository=trump_bond')
# print(response.status_code, response) 
response = requests.post('http://127.0.0.1:5000/dna/v1/repositories?repository=fl_abortion')
print(response.status_code, response) 
response = requests.post('http://127.0.0.1:5000/dna/v1/repositories?repository=trump_trial')
print(response.status_code, response) 
# Verify that the repositories were created
response = requests.get('http://127.0.0.1:5000/dna/v1/repositories')
print(response.status_code, response.json()) 

201 <Response [201]>
201 <Response [201]>
200 [{'created': '2024-05-14T23:58:06', 'repository': 'fl_abortion'}, {'created': '2024-05-14T23:58:07', 'repository': 'trump_trial'}]


In [4]:
# Articles for ingest
ceasefire_aljazeera = {
            "title": "UN Security Council demands immediate Gaza ceasefire as US abstains",
            "published": "2024-03-25T00:00:00",
            "url": "https://www.aljazeera.com/news/2024/3/25/un-security-council-adopts-resolution-calling-for-immediate-gaza-ceasefire"}
ceasefire_breitbart = {
            "title": "GOP Blasts Biden’s ‘Unconscionable’ U.N. Vote Abstention on Gaza",
            "published": "2024-03-25T00:00:00",
            "url": "https://www.breitbart.com/politics/2024/03/25/gop-blasts-bidens-unconscionable-un-vote-abstention-gaza-anti-israel-president/"}
ceasefire_cnn = {
            "title": "Israel cancels Washington visit after US allows UN Gaza ceasefire resolution to pass",
            "published": "2024-03-25T00:00:00",
            "url": "https://www.cnn.com/2024/03/25/middleeast/un-security-council-gaza-israel-ceasefire-intl/index.html"}
ceasefire_fox = {
            "title": "Netanyahu cancels Israeli delegation's trip to Washington after US fails to veto UN cease-fire resolution",
            "published": "2024-03-25T00:00:00",
            "url": "https://www.foxnews.com/politics/netanyahu-cancels-israeli-delegations-trip-washington-us-fails-veto-un-cease-fire-resolution"}
ceasefire_huffpost = {
            "title": "UN Demand For Gaza Cease-fire Provokes Strongest Clash Between US And Israel Since War Began",
            "published": "2024-03-25T00:00:00",
            "url": "https://www.huffpost.com/entry/un-cease-fire-gaza_n_66019107e4b08460c3e7dc60"}
ceasefire_nyt = {
            "title": "U.N. Security Council Calls for Immediate Cease-Fire in Gaza as U.S. Abstains",
            "published": "2024-03-25T00:00:00",
            "url": "https://www.nytimes.com/2024/03/25/world/middleeast/un-security-council-gaza-ceasefire.html?searchResultPosition=2"}
ceasefire_watimes = {
            "title": "White House defends its refusal to block U.N. cease-fire resolution",
            "published": "2024-03-25T00:00:00",
            "url": "https://www.washingtontimes.com/news/2024/mar/25/white-house-defends-its-refusal-to-block-un-cease-/"}
ceasefire_wsj = {
            "title": "Israel Cancels Visit to Washington After U.S. Abstains on U.N. Cease-Fire Resolution",
            "published": "2024-03-25T00:00:00",
            "url": "https://www.wsj.com/world/middle-east/hostage-talks-stall-as-israeli-defense-minister-arrives-in-washington-ea36821d"}

fl_abortion_aljazeera = {
            "title": "Biden denounces Florida abortion ruling as ‘outrageous’ as state vote looms",
            "published": "2024-04-02T00:00:00",
            "url": "https://www.aljazeera.com/news/2024/4/2/biden-denounces-florida-abortion-ruling-as-outrageous-as-state-vote-looms"}
fl_abortion_breitbart = {
            "title": "Florida Supreme Court Upholds 15-Week Abortion Limit, But Voters Will Decide in November",
            "published": "2024-03-25T00:00:00",
            "url": "https://www.breitbart.com/politics/2024/04/01/florida-supreme-court-upholds-15-week-abortion-limit-but-voters-will-decide-in-november/"}
fl_abortion_cnn = {
            "title": "Florida six-week abortion ban will soon become law, but voters will consider a constitutional amendment this fall",
            "published": "2024-04-02T00:00:00",
            "url": "https://www.cnn.com/2024/04/01/politics/florida-six-week-abortion-ban/index.html"}
fl_abortion_fox = {
            "title": "Florida Supreme Court approves abortion ballot initiative while upholding 15-week ban",
            "published": "2024-03-25T00:00:00",
            "url": "https://www.foxnews.com/politics/florida-supreme-court-approves-abortion-ballot-initiative-upholding-15-week-ban"}
fl_abortion_huffpost = {
            "title": "Top Florida Court Rules Against Protections For Abortion, Greenlights 6-Week Ban",
            "published": "2024-03-25T00:00:00",
            "url": "https://www.huffpost.com/entry/florida-court-abortion-6-week-ban_n_65088b87e4b0f04446230d7e"}
fl_abortion_nyt = {
            "title": "Florida Court Allows 6-Week Abortion Ban, but Voters Will Get to Weigh In",
            "published": "2024-03-25T00:00:00",
            "url": "https://www.nytimes.com/2024/04/01/us/florida-abortion-law-supreme-court.html"}
fl_abortion_watimes = {
            "title": "Florida Supreme Court upholds state’s 15-week ban on most abortions, paving way for 6-week ban",
            "published": "2024-03-25T00:00:00",
            "url": "https://www.washingtontimes.com/news/2024/apr/1/florida-supreme-court-upholds-states-15-week-ban-2/"}
fl_abortion_wsj = {
            "title": "Florida Abortion Rulings to Keep Issue Front and Center in November",
            "published": "2024-03-25T00:00:00",
            "url": "https://www.wsj.com/politics/elections/florida-abortion-rulings-to-keep-issue-front-and-center-in-november-1168511c"}

trump_bond_aljazeera = {
            "title": "New York court lowers $454m bond as Trump appeals civil fraud case",
            "published": "2024-03-25T00:00:00",
            "url": "https://www.aljazeera.com/news/2024/3/25/new-york-court-pauses-collection-of-454m-in-trump-civil-fraud-case"}
trump_bond_breitbart = {
            "title": "‘Partisan Hacks’: Conservatives React to Trump Bond Ruling",
            "published": "2024-03-25T00:00:00",
            "url": "https://www.breitbart.com/politics/2024/03/25/partisan-hacks-conservatives-react-trump-bond-ruling/"}
trump_bond_cnn = {
            "title": "Trump bond lowered to $175 million as he appeals civil fraud judgment in New York",
            "published": "2024-03-25T00:00:00",
            "url": "https://www.cnn.com/2024/03/25/politics/trump-civil-fraud-trial-bond/index.html"}
trump_bond_fox = {
            "title": "Trump's $454M judgment bond slashed by more than half in appeals court ruling",
            "published": "2024-03-25T00:00:00",
            "url": "https://www.foxnews.com/politics/trumps-454m-judgment-bond-slashed-more-than-half-appeals-court-ruling"}
trump_bond_huffpost = {
            "title": "Judge Reduces Donald Trump's Bond In Civil Fraud Appeal To $175 Million",
            "published": "2024-03-25T00:00:00",
            "url": "https://www.huffpost.com/entry/trump-bond-reduced-fraud-appeal_n_660198c0e4b059750143d44c"}
trump_bond_nyt = {
            "title": "Trump’s Bond in Civil Fraud Case Is Reduced to $175 Million",
            "published": "2024-03-25T00:00:00",
            "url": "https://www.nytimes.com/2024/03/25/nyregion/trump-bond-reduced.html"}
trump_bond_watimes = {
            "title": "Trump can post smaller bond amount of $175M within 10 days, appeals court rules",
            "published": "2024-03-25T00:00:00",
            "url": "https://www.washingtontimes.com/news/2024/mar/25/donald-trump-can-post-smaller-bond-amount-of-175m-/"}
trump_bond_wsj = {
            "title": "Trump Can Post Smaller $175 Million Bond in Civil Fraud Case",
            "published": "2024-03-25T00:00:00",
            "url": "https://www.wsj.com/us-news/law/donald-trump-new-york-fraud-bond-due-05f590e4"}

trump_trial_aljazeera = {
            "title": "Six takeaways from first day of Trump’s New York hush money criminal trial",
            "published": "2024-04-15T00:00:00",
            "url": "https://www.aljazeera.com/news/2024/4/15/five-takeaways-from-first-day-of-trumps-historic-new-york-cirminal-trial"}
trump_trial_breitbart = {
            "title": "Unprecedented Trump Criminal Trial Begins with Jury Selection",
            "published": "2024-04-15T00:00:00",
            "url": "https://www.breitbart.com/politics/2024/04/15/unprecedented-trump-criminal-trial-begins-with-jury-selection/"}
trump_trial_cnn = {
            "title": "Trump has an unusual request on trial Day 1",
            "published": "2024-04-15T00:00:00",
            "url": "https://www.cnn.com/2024/04/15/politics/trump-trial-day-1-what-matters/index.html"}
trump_trial_fox = {
            "title": "Trump hush money trial begins in New York City with jury selection",
            "published": "2024-04-15T00:00:00",
            "url": "https://www.foxnews.com/politics/trump-bragg-hush-money-trial-begins-new-york-city-jury-selection"}
trump_trial_huffpost = {
            "title": "Manhattan Court To Find Jurors To Hear First-Ever Criminal Case Against An Ex-President",
            "published": "2024-04-12T00:00:00",
            "url": "https://www.huffpost.com/entry/trump-bond-reduced-fraud-appeal_n_660198c0e4b059750143d44c"}
trump_trial_nyt = {
            "title": "A Historic Day",
            "published": "2024-04-15T00:00:00",
            "url": "https://www.nytimes.com/2024/04/15/us/politics/trump-trial-manhattan-first-day.html"}
trump_trial_watimes = {
            "title": "Trump’s history-making hush-money trial begins with challenge of picking a jury",
            "published": "2024-04-15T00:00:00",
            "url": "https://www.washingtontimes.com/news/2024/apr/15/trumps-hush-money-trial-begins-with-challenge-of-p/"}
trump_trial_wsj = {
            "title": "Donald Trump Faces Deep Blue Jury Pool as Hush-Money Case Begins",
            "published": "2024-04-15T00:00:00",
            "url": "https://www.wsj.com/us-news/law/trump-hush-money-trial-jury-2bc58769"}

# repos = ['ceasefire', 'fl_abortion', 'trump_bond', 'trump_trial']
repos = ['fl_abortion', 'trump_trial']
# 2 ~neutral sources (Al Jazeera, WSJ), 3 right/conservative sources(Breitbart, Fox, WA Times)
#    3 left/liberal sources (CNN, Huffington Post, NYT)
sources = {'aljazeera': 'Al Jazeera',    # ~ neutral
           'wsj': 'Wall Street Journal',
           'breitbart': 'Breitbart',     # conservative
           'fox': 'Fox News',
           'watimes': 'Washington Times',
           'cnn': 'CNN',                 # liberal
           'huffpost': 'Huffington Post',
           'nyt': 'New York Times'}

In [5]:
# Function to ingest the first 10 sentences of the 8 news articles regarding a specific topic 
# (ingested to the 'topic' repository)
def ingest_articles(repo: str): 
    for source in sources:
        repo_source = f'{repo}_{source}'
        print('repo_source ', repo_source)
        with open(f'articles/{repo_source}.txt', 'r') as news:
            news_text = news.read()  
        metadata = globals()[repo_source]
        req_dict = metadata
        req_dict["source"] = sources[source]
        req_dict["text"] = news_text
        response = requests.post(
            f'http://127.0.0.1:5000/dna/v1/repositories/narratives?repository={repo}&sentences=10',
            json=req_dict)
        print(response)
        print(response.status_code, response.json())
        print()

In [6]:
ingest_articles('fl_abortion')

ingest_articles('trump_trial')

repo_source  fl_abortion_aljazeera
<Response [201]>
201 {'narrativeDetails': {'narrativeId': 'e5c67042', 'narrativeMetadata': {'published': '2024-04-02T00:00:00', 'source': 'Al Jazeera', 'title': 'Biden denounces Florida abortion ruling as ‘outrageous’ as state vote looms', 'url': 'https://www.aljazeera.com/news/2024/4/2/biden-denounces-florida-abortion-ruling-as-outrageous-as-state-vote-looms'}, 'numberIngested': 10, 'numberOfSentences': 35, 'numberOfTriples': '631', 'processed': '2024-05-15T00:06:46'}, 'repository': 'fl_abortion'}

repo_source  fl_abortion_wsj
<Response [201]>
201 {'narrativeDetails': {'narrativeId': 'b71a8e2a', 'narrativeMetadata': {'published': '2024-03-25T00:00:00', 'source': 'Wall Street Journal', 'title': 'Florida Abortion Rulings to Keep Issue Front and Center in November', 'url': 'https://www.wsj.com/politics/elections/florida-abortion-rulings-to-keep-issue-front-and-center-in-november-1168511c'}, 'numberIngested': 10, 'numberOfSentences': 36, 'numberOfTriples

In [7]:
# Check what articles are in the fl_abortion repository 
response = requests.get('http://127.0.0.1:5000/dna/v1/repositories/narratives?repository=fl_abortion')
print(response.status_code, json.dumps(response.json(), indent=1))

200 {
 "narratives": [
  {
   "narrativeId": "e5c67042",
   "narrativeMetadata": {
    "published": "2024-04-02T00:00:00",
    "source": "Al Jazeera",
    "title": "Biden denounces Florida abortion ruling as \u2018outrageous\u2019 as state vote looms",
    "url": "https://www.aljazeera.com/news/2024/4/2/biden-denounces-florida-abortion-ruling-as-outrageous-as-state-vote-looms"
   },
   "numberIngested": "10",
   "numberOfSentences": "35",
   "numberOfTriples": "631",
   "processed": "2024-05-15T00:06:46"
  },
  {
   "narrativeId": "b71a8e2a",
   "narrativeMetadata": {
    "published": "2024-03-25T00:00:00",
    "source": "Wall Street Journal",
    "title": "Florida Abortion Rulings to Keep Issue Front and Center in November",
    "url": "https://www.wsj.com/politics/elections/florida-abortion-rulings-to-keep-issue-front-and-center-in-november-1168511c"
   },
   "numberIngested": "10",
   "numberOfSentences": "36",
   "numberOfTriples": "707",
   "processed": "2024-05-15T00:15:51"
  },


In [8]:
# In the trump_trial repository
response = requests.get('http://127.0.0.1:5000/dna/v1/repositories/narratives?repository=trump_trial')
print(response.status_code, json.dumps(response.json(), indent=1))

200 {
 "narratives": [
  {
   "narrativeId": "53bfa4c1",
   "narrativeMetadata": {
    "published": "2024-04-15T00:00:00",
    "source": "Al Jazeera",
    "title": "Six takeaways from first day of Trump\u2019s New York hush money criminal trial",
    "url": "https://www.aljazeera.com/news/2024/4/15/five-takeaways-from-first-day-of-trumps-historic-new-york-cirminal-trial"
   },
   "numberIngested": "10",
   "numberOfSentences": "66",
   "numberOfTriples": "509",
   "processed": "2024-05-15T01:08:25"
  },
  {
   "narrativeId": "5f98b375",
   "narrativeMetadata": {
    "published": "2024-04-15T00:00:00",
    "source": "Wall Street Journal",
    "title": "Donald Trump Faces Deep Blue Jury Pool as Hush-Money Case Begins",
    "url": "https://www.wsj.com/us-news/law/trump-hush-money-trial-jury-2bc58769"
   },
   "numberIngested": "10",
   "numberOfSentences": "40",
   "numberOfTriples": "547",
   "processed": "2024-05-15T01:15:43"
  },
  {
   "narrativeId": "70eada27",
   "narrativeMetadata"

In [15]:
# Dump of the article meta-data and triples
for repo in ('fl_abortion', 'trump_trial'):
    response = requests.get(f'http://127.0.0.1:5000/dna/v1/repositories/narratives?repository={repo}')
    narr_details = response.json()['narratives']
    narr_ids = []
    for detail in narr_details:
        narr_ids.append(detail['narrativeId'])
    for narr_id in narr_ids:
        response = requests.get(
            f'http://127.0.0.1:5000/dna/v1/repositories/narratives/graphs?repository={repo}&narrativeId={narr_id}')
        resp_json = response.json()
        with open(f'dumps/{repo}_{narr_id}_details.json', 'w') as detail_out:
            detail_out.write(json.dumps(dict(resp_json['narrativeDetails'])))
        with open(f'dumps/{repo}_{narr_id}.ttl', 'w') as triples_out:
            for triple in resp_json['triples']:
                triples_out.write(triple)
    